In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [3]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

# Read Data

In [4]:
rev_kp_dataset_df = pd.read_pickle("../data/training/train_data.pkl")

In [5]:
rev_kp_dataset_df

,index,level_1,domain,categories_list,categories,name,business_id,sentences,opinions_x,aspects_x,...,opinion_phrases_y,key_point,predicted_WA_y,label,aspects_similarity,aspects_x_len,aspects_y_len,id_pair,sentiments_x,sentiments_y
1,487383,2,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Free WiFi in lobby,[free],[wifi],...,[very fast wifi],Wifi was very fast ~ 20MBps .,0.5944222211837769,1.0,NaN,NaN,NaN,487383_2##702558_5,[positive],[positive]
3,487385,0,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Been staying at this hotel for years now and still enjoy the place .,[enjoy],[place],...,[family friendly place],"Relaxed , quiet , family friendly place .",0.5604457855224609,1.0,NaN,NaN,NaN,487385_0##777430_1,[positive],[positive]
4,487385,0,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Been staying at this hotel for years now and still enjoy the place .,[enjoy],[place],...,[family oriented place],Definitely a family oriented place .,0.43907630443573,1.0,NaN,NaN,NaN,487385_0##511294_4,[positive],[positive]
27,487386,3,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Staff was friendly .,[friendly],[staff],...,[above and beyond staff],Staff goes above and beyond .,0.5060063004493713,1.0,NaN,NaN,NaN,487386_3##922097_1,[positive],[positive]
28,487386,3,Hotels,"['Event Planning & Services', 'Venues & Event Spaces', 'Hotels', 'Hotels & Travel']","Event Planning & Services, Venues & Event Spaces, Hotels, Hotels & Travel",Sheraton Centre Toronto Hotel,M7FOXsuEuRbwEfxUrNKv5w,Staff was friendly .,[friendly],[staff],...,[amazing staff],Amazing staff & customer service .,0.5119611024856567,1.0,NaN,NaN,NaN,487386_3##884704_1,[positive],[positive]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50051,880945,1,Arts & Entertainment,"['Performing Arts', 'Arts & Entertainment']","Performing Arts, Arts & Entertainment",Rock of Ages,_F3AMoo_zdl-he384ISQbw,Ushers are all standby to guide you to the seats .,[standby],[ushers],...,[friendly and attentive concessions],Concessions are friendly and attentive .,0.48531198501586914,0.0,NaN,NaN,NaN,880945_1##999990_2,[negative],[positive]
50052,929246,2,Arts & Entertainment,"['Cinema', 'Arts & Entertainment']","Cinema, Arts & Entertainment",Cinemark Mesa 16,lqCJiHlxzRpuYt-u_XQUvg,Management was horrible and I 'm beyond disappointed with how rude and unprofessional they were !,[horrible],[management],...,[friendly and attentive concessions],Concessions are friendly and attentive .,0.48531198501586914,0.0,NaN,NaN,NaN,929246_2##999990_2,[negative],[positive]
50053,901588,0,Arts & Entertainment,"['Casinos', 'Nightlife', 'Arts & Entertainment', 'Bars', 'Adult Entertainment']","Casinos, Nightlife, Arts & Entertainment, Bars, Adult Entertainment",Playboy Club,3g8zzg_5__kXMUM-8F6V1w,Sexy dealers .,[sexy],[dealers],...,[friendly and attentive concessions],Concessions are friendly and attentive .,0.48531198501586914,0.0,NaN,NaN,NaN,901588_0##999990_2,[positive],[positive]
50054,842226,3,Arts & Entertainment,"['Arts & Entertainment', 'Arcades', 'Active Life', 'Venues & Event Spaces', 'Party & Event Planning', 'Amusement Parks', 'Event Planning & Services', 'Mini Golf', 'Laser Tag']","Arts & Entertainment, Arcades, Active Life, Venues & Event Spaces, Party & Event Planning, Amusement Parks, E

In [6]:
rev_kp_dataset_df = rev_kp_dataset_df.rename(columns={'attributes': 'topic', 'sentences': 'text'})
rev_kp_dataset_df['topic'] = rev_kp_dataset_df['topic'].apply(lambda x: ','.join(x))

# Prepare Contrastive Input

In [7]:
rev_kp_dataset_df = rev_kp_dataset_df.rename(columns={'text': 'full_comment', 'key_point': 'full_keypoint'})

In [8]:
def prepare_contrastive_input(row):
    return row.iloc[0][0] + ' <SEP> ' + row.iloc[1][0]

In [9]:
rev_kp_dataset_df.columns

Index(['index', 'level_1', 'domain', 'categories_list', 'categories', 'name',
       'business_id', 'full_comment', 'opinions_x', 'aspects_x',
       'opinion_phrases_x', 'sid', 'topic', 'opinion_aspect_pos', 'aspect_pos',
       'num_of_token', 'predicted_WA_x', 'id_x', 'id_y', 'aspects_y',
       'opinions_y', 'opinion_phrases_y', 'full_keypoint', 'predicted_WA_y',
       'label', 'aspects_similarity', 'aspects_x_len', 'aspects_y_len',
       'id_pair', 'sentiments_x', 'sentiments_y'],
      dtype='object')

In [10]:
rev_kp_dataset_df['comment'] = rev_kp_dataset_df[['opinion_phrases_x', 'sentiments_x']].apply(prepare_contrastive_input, axis=1)
rev_kp_dataset_df['comment']

1                      free wifi <SEP> positive
3                    enjoy place <SEP> positive
4                    enjoy place <SEP> positive
27                friendly staff <SEP> positive
28                friendly staff <SEP> positive
                          ...                  
50051             standby ushers <SEP> negative
50052        horrible management <SEP> negative
50053               sexy dealers <SEP> positive
50054     freaking awesome staff <SEP> positive
50055    very entertaining staff <SEP> positive
Name: comment, Length: 58289, dtype: object

In [11]:
rev_kp_dataset_df['keypoint'] = rev_kp_dataset_df[['opinion_phrases_y', 'sentiments_y']].apply(prepare_contrastive_input, axis=1)
rev_kp_dataset_df['keypoint']

1                            very fast wifi <SEP> positive
3                     family friendly place <SEP> positive
4                     family oriented place <SEP> positive
27                   above and beyond staff <SEP> positive
28                            amazing staff <SEP> positive
                               ...                        
50051    friendly and attentive concessions <SEP> positive
50052    friendly and attentive concessions <SEP> positive
50053    friendly and attentive concessions <SEP> positive
50054    friendly and attentive concessions <SEP> positive
50055    friendly and attentive concessions <SEP> positive
Name: keypoint, Length: 58289, dtype: object

# Train-Dev Split

In [12]:
dataset = Dataset.from_pandas(rev_kp_dataset_df[['topic', 'comment', 'full_comment',
                                                 'keypoint', 'full_keypoint', 
                                                 'domain', 'label']]).shuffle(seed=42)

In [13]:
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
## Train/Val Split by group
groups = dataset['topic']
gss = GroupShuffleSplit(n_splits=1, train_size=0.85, random_state=32)

for train_idx, val_idx in gss.split(dataset, groups=groups):
    dataset.set_format(None)
    
    # Define data loaders for training and testing data in this fold
    train_dataset = Dataset.from_dict(dataset[train_idx])
    val_dataset = Dataset.from_dict(dataset[val_idx])
    
    train_dataset.set_format("pandas")
    d = train_dataset[:]
    print("TRAIN CLASSES: ", len(d['topic'].unique()))
    train_dataset.set_format(None)

    val_dataset.set_format("pandas")
    d = val_dataset[:]
    print("VAL CLASSES: ", len(d['topic'].unique()))
    val_dataset.set_format(None)

TRAIN CLASSES:  11
VAL CLASSES:  2


In [15]:
datasets = DatasetDict({
    "train": train_dataset,
    "valid": val_dataset})
datasets

DatasetDict({
    train: Dataset({
        features: ['topic', 'comment', 'full_comment', 'keypoint', 'full_keypoint', 'domain', 'label', '__index_level_0__'],
        num_rows: 56721
    })
    valid: Dataset({
        features: ['topic', 'comment', 'full_comment', 'keypoint', 'full_keypoint', 'domain', 'label', '__index_level_0__'],
        num_rows: 1568
    })
})

In [16]:
datasets.set_format("pandas")

In [17]:
training_df = datasets['train'][:]
valid_df = datasets['valid'][:]

In [18]:
training_df

,topic,comment,full_comment,keypoint,full_keypoint,domain,label,__index_level_0__
0,staff,happy store <SEP> positive,I 'm very happy with this store .,commendable patience <SEP> positive,His patience was commendable .,Automotive,0.0,44352
1,food -> quality,satisfactory massage <SEP> positive,My husband found his massage satisfactory,best pedicure <SEP> positive,The best pedicure / massages .,Beauty & Spas,0.0,459
2,value-for-money,better prices <SEP> negative,But there are many others in the area with better prices .,too expensive restaurant <SEP> negative,Restaurant is much too expensive .,Hotels,0.0,20746
3,staff,could not be any better service <SEP> negative,Service could not be any better from all the staff .,attentively managed textures <SEP> positive,"Colors , textures were attentively managed .",Restaurants,0.0,6718
4,wait-time,50/50 services <SEP> negative,Services was 50/50 .,always on point service <SEP> positive,Service is always on point .,Restaurants,0.0,12860
...,...,...,...,...,...,...,...,...
56716,wait-time,slow check in <SEP> negative,Check in was slow with just one person sitting at the front desk with several people in line at peak check in time .,heavily congested elevators <SEP> negative,Elevators were heavily congested .,Hotels,0.0,26724
56717,staff,amazing service <SEP> positive,Amazing service .,lovely service <SEP> positive,"Terrific variety , lovely service !",Restaurants,1.0,97244
56718,food -> quality,always been food <SEP> positive,The food has always been consistently good .,consistently quality food <SEP> positive,Consistently quality food and margaritas ! ! !,Restaurants,1.0,109821
56719,restaurant -> atmosphere,great flavor selection <SEP> positive,Great flavor selection .,crisp wings <SEP> positive,Wings were crisp as requested .,Restaurants,0.0,13880


In [19]:
valid_df

,topic,comment,full_comment,keypoint,full_keypoint,domain,label,__index_level_0__
0,drink -> alcohol,yum root beer float <SEP> positive,"Black cow ( root beer float with chocolate sauce- yum ! ) and a loaded rail burger with a fried egg and house made chips , he had some kind of burger with bacon and three kinds of onions plus onion rings .",poor beer selection <SEP> negative,Beer selection is poor .,Restaurants,0.0,6072
1,restaurant -> comfort,empty hotel <SEP> negative,The hotel was pretty much empty when we came .,understaffed front desk <SEP> negative,The front desk is understaffed .,Hotels,0.0,28871
2,restaurant -> comfort,ping pong pool tables <SEP> positive,Pool tables ping pong .,comfy chairs <SEP> positive,Comfy chairs and couches .,Hotels,0.0,20960
3,restaurant -> comfort,comfy couch <SEP> positive,It was comfy for a pull out couch .,comfy beds <SEP> positive,"beds were comfy ,",Hotels,0.0,44266
4,restaurant -> comfort,plenty of reserved seating <SEP> positive,"They have online reserved seating , plenty of theatres for movie times , and the seats recline .",assigned seating <SEP> positive,I love the assigned seating .,Arts & Entertainment,1.0,48404
...,...,...,...,...,...,...,...,...
1563,drink -> alcohol,pricey beer <SEP> positive,The beer is just as pricey as any of those other places unless you want a Miller Lite ( no thanks ) .,great apple cider <SEP> positive,Great apple cider and donuts .,Arts & Entertainment,0.0,31552
1564,restaurant -> comfort,plenty of reserved seating <SEP> positive,"They have online reserved seating , plenty of theatres for movie times , and the seats recline .",comfy recliners <SEP> positive,The recliners are so comfy,Arts & Entertainment,0.0,37348
1565,restaurant -> comfort,very comfortable place <SEP> positive,Very comfortable place to hang .,super comfortable seats <SEP> positive,Super comfortable seats .,Arts & Entertainment,0.0,40485
1566,drink -> alcohol,delicious apple ciders <SEP> positive,Those apple cider donuts and apple ciders were delicious !,great apple cider <SEP> positive,Great apple cider and donuts .,Arts & Entertainment,0.0,31557


In [20]:
training_df['label'] = training_df.label.apply(lambda x: int(x))
training_df[['comment', 'keypoint', 'domain', 'label']].to_csv('./training_material/siamese-data/training_df_contrastive.csv')

valid_df['label'] = valid_df.label.apply(lambda x: int(x))
valid_df[['comment', 'keypoint', 'domain', 'label']].to_csv('./training_material/siamese-data/valid_df_contrastive.csv')

## Create separate sets of comments/KPs

In [21]:
contrastive_datasets = datasets
contrastive_datasets

DatasetDict({
    train: Dataset({
        features: ['topic', 'comment', 'full_comment', 'keypoint', 'full_keypoint', 'domain', 'label', '__index_level_0__'],
        num_rows: 56721
    })
    valid: Dataset({
        features: ['topic', 'comment', 'full_comment', 'keypoint', 'full_keypoint', 'domain', 'label', '__index_level_0__'],
        num_rows: 1568
    })
})

In [22]:
contrastive_datasets = contrastive_datasets.rename_columns({'full_keypoint': 'full_key_point', 'keypoint': 'key_point'})

In [23]:
# Functions to label id
i = 0
curr_topic = ""
def label_group_id(grp_df):
    global i
    grp_df['group_id'] = i
    grp_df = grp_df.reset_index(drop=True)
    grp_df = grp_df.reset_index()
    i += 1
    return grp_df

def label_kp_id(grp_df):
    global i
    global curr_topic
    if grp_df['topic'].iloc[0] != curr_topic:
        i = 0
        curr_topic = grp_df['topic'].iloc[0]
    grp_df['kp_id'] = i
    grp_df = grp_df.reset_index(drop=True)
    i += 1
    return grp_df

def label_comment_id(grp_df):
    global i
    global curr_topic
    if grp_df['topic'].iloc[0] != curr_topic:
        i = 0
        curr_topic = grp_df['topic'].iloc[0]
    grp_df['comm_id'] = i
    grp_df = grp_df.reset_index(drop=True)
    i += 1
    return grp_df

In [24]:
def create_comment_kp_label_dfs(datasets, subset):
    dataset = datasets[subset].shuffle(seed=42)
    dataset.set_format("pandas")
    df = dataset[:]
    df = df.rename(columns={'text': 'comment'})
    
    global i
    i = 0
    df = df.groupby(['topic']).apply(label_group_id).reset_index(drop=True)  
    
    i = 0
    df = df.groupby(['topic', 'comment']).apply(label_comment_id).reset_index(drop=True)
    
    i = 0
    df = df.groupby(['topic', 'key_point']).apply(label_kp_id).reset_index(drop=True)
    
    df['comment_id'] = "comment_" + df['group_id'].astype(str) + "_" + df['comm_id'].astype(str)
    df['key_point_id'] = "kp_" + df['group_id'].astype(str) + "_" + df['kp_id'].astype(str)
    
    comments_df = df[:].sort_values(by=['group_id', 'comm_id'])\
        [['comment_id', 'comment', 'full_comment', 'topic', 'domain']].\
        drop_duplicates().reset_index(drop=True)
    
    keypoints_df = df[:].sort_values(by=['group_id', 'kp_id'])\
        [['key_point_id', 'key_point', 'full_key_point', 'topic', 'domain']].\
        drop_duplicates().reset_index(drop=True)
    
    labels_df = df[:].sort_values(by=['group_id', 'index'])[['comment_id', 'key_point_id', 'label']]
    labels_df['label'] = labels_df.label.apply(lambda x: int(x))
    
    return comments_df.reset_index(drop=True), keypoints_df.reset_index(drop=True), labels_df.reset_index(drop=True)

In [25]:
for subset, subset_save in zip(['train', 'valid'], ['train', 'dev']):
    comments_df, keypoints_df, labels_df = create_comment_kp_label_dfs(contrastive_datasets, subset)
    comments_df.to_csv(f"./training_material/siamese-data/comments_{subset_save}.csv", index=False)
    keypoints_df.to_csv(f"./training_material/siamese-data/key_points_{subset_save}.csv", index=False)
    labels_df.to_csv(f"./training_material/siamese-data/labels_{subset_save}.csv", index=False)

In [26]:
comments_df

,comment_id,comment,full_comment,topic,domain
0,comment_0_0,18 holes course <SEP> negative,"The course itself is 18 holes , some a little more challenging making it more competitive .",drink -> alcohol,Arts & Entertainment
1,comment_0_1,always a blast mini golf <SEP> positive,Mini golf is always a blast but having it glow in the dark makes it that much more fun .,drink -> alcohol,Arts & Entertainment
2,comment_0_2,always good beer <SEP> positive,OB beer always good !,drink -> alcohol,Restaurants
3,comment_0_3,amazing shrimp cocktail <SEP> positive,The Shrimp cocktail is amazing !,drink -> alcohol,Restaurants
4,comment_0_4,among the best spring rolls <SEP> positive,"Their spring rolls are among the best I 've ever had , though a little on the greasy side .",drink -> alcohol,Restaurants
...,...,...,...,...,...
385,comment_1_246,wider seats <SEP> positive,The seats are wider and more padded .,restaurant -> comfort,Arts & Entertainment
386,comment_1_247,wonderful seating <SEP> positive,"Seating is wonderful , prices ca n't be beat , and its surrounded by a few resturants and shops so you could catch dinner and do it all in one trip !",restaurant -> comfort,Arts & Entertainment
387,comment_1_248,wonderful table <SEP> positive,We had a wonderful table right by the window .,restaurant -> comfort,Restaurants
388,comment_1_249,worn bed <SEP> negative,The bed is worn and the couch bed smells like urine .,restaurant -> comfort,Hotels
